<a href="https://colab.research.google.com/github/dasd412/my-first-ai/blob/main/%08k_mean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# k 평균 군집 알고리즘은 각 과일 사진의 평균 값을 자동으로 찾아준다.
# 이 평균값이 클러스터 중심에 위치하기 때문에 클러스터 중심 또는 센트로이드라고 부른다.

# k 평균 알고리즘 작동 방식 (처음에는 랜덤하게 클러스터 중심을 선택해서 점차 가까운 샘플의 중심으로 이동하는 알고리즘이다.)
# 1. 무작위로 k개의 클러스터 중심을 정한다.
# 2. 각 샘플에서 가장 가까운 클러스터 중심을 찾아 해당 클러스터의 샘플로 지정한다.
# 3. 클러스터에 속한 샘플의 평균값으로 클러스터 중심을 변경한다.
# 4. 클러스터 중심에 변화가 없을 때까지 2번으로 돌아가 반복한다.

In [ ]:
# 데이터 다운로드
!wget https://bit.ly/fruits_300_data -O fruits_300.npy

In [ ]:
import numpy as np

fruits=np.load('fruits_300.npy')

# (샘플 개수, 너비, 높이) 크기의 3차원 배열을 (샘플 개수, 너비 * 높이)의 2차원 배열로 변경
fruits_2d=fruits.reshape(-1,100*100)

In [ ]:
# 클러스터 개수를 3으로 지정함
from sklearn.cluster import KMeans

km=KMeans(n_clusters=3,random_state=42)
km.fit(fruits_2d)

In [ ]:
# 군집된 결과는 KMeans 객체의 labels_ 속성에 저장됨. labels_ 배열의 길이는 샘플 개수와 같음 (해당 배열은 각 샘플이 어떤 레이블에 해당하는 지 나타냄)
print(km.labels_)

In [ ]:
print(np.unique(km.labels_,return_counts=True))
# 첫 번째 클러스터 (레이블 0)이 111개의 샘플, 두 번쨰 클러스터 (레이블 1)이 98개의 샘플, 세 번째 클러스터 (레이블 2)가 91개의 샘플을 모은것을 확인할 수 있다.

In [ ]:
# 각 클러스터가 어떤 이미지를 나타냈는지 그림으로 출력하기 위해 유틸 함수 만듬
import matplotlib.pyplot as plt

def draw_fruits(arr,ratio=1):
    n=len(arr)
    # 한줄에 10개씩 이미지를 그린다. 샘플 개수를 10으로 나누어 전체 행 개수 계산
    rows=int(np.ceil(n/10))
    # 행이 1개이면 열의 개수는 샘플 개수. 그렇지 않으면 10개
    cols = n if rows<2 else 10

    fig,axs=plt.subplots(rows,cols,figsize=(cols*ratio,rows*ratio),squeeze=False)

    for i in range(rows):
        for j in range(cols):
            if i*10+j < n :# n개까지만 그림.
                axs[i,j].imshow(arr[i*10+j],cmap='gray_r')
            axs[i,j].axis('off')
    plt.show()

In [ ]:
# 넘파이 불리언 인덱싱을 활용하여 레이블의 0인 과일 사진만 그려보기
draw_fruits(fruits[km.labels_==0])

In [ ]:
draw_fruits(fruits[km.labels_==1])

In [ ]:
draw_fruits(fruits[km.labels_==2])

k mean 알고리즘이 완벽히 구별해낸 것은 아니지만, 훈련 데이터에 타깃 레이블을 전혀 제공하지 않았음에도 스스로 비슷한 샘플을 잘 모았다.

In [ ]:
# KMeans 클래스가 최종적으로 찾은 클러스터 중심은 cluster_centers_ 속성에 저장되어 있다.
# 2차원으로 압축된 걸 풀어야 하므로 100*100의 2차원 배열로 바꿔서 표현해야 함
draw_fruits(km.cluster_centers_.reshape(-1,100,100),ratio=3)

In [ ]:
# 훈련 데이터 샘플에서 클러스터 중심까지 거리로 변환해주는 transform()
print(km.transform(fruits_2d[100:101]))

In [ ]:
print(km.predict(fruits_2d[100:101]))

In [ ]:
draw_fruits(fruits[100:101])

In [ ]:
# k means 알고리즘이 반복한 횟수
print(km.n_iter_)

In [ ]:
# 사실 위 예제는 클러스터 개수를 맘대로 지정했기 때문에 타깃에 대한 정보를 활용한 것이다.
# 아예 최적의 k를 찾는 방법이 필요하다. (실무에선)

In [ ]:
# 적절한 클러스터 개수 k를 찾는 방법으로 엘보우가 있다.
# 엘보우는 클러스터 중심과 클러스터에 속한 샘플 사이의 거리 제곱합인 '이너셔'의 변화를 관찰한다. 그리고 최적의 클러스터 개수를 찾아나간다.
# 클러스터 개수와 이너셔 값의 그래프가 꺽이는 지점이 최적의 클러스터 개수다.

In [ ]:
inertia=[]

for k in range(2,7):
    km=KMeans(n_clusters=k,random_state=42)
    km.fit(fruits_2d)
    inertia.append(km.inertia_)

plt.plot(range(2,7),inertia)
plt.xlabel('k')
plt.ylabel('inertia')
plt.show()